# Raw

In [ ]:
import requests
import re
import html
import json
from bs4 import BeautifulSoup
import pandas as pd


In [11]:
def getPostInfosFromContent(content):
    jsonDataArray = re.findall(r'\{\"result\"\:\{\"data.*?,\"variables', content)
    #ex. {"result": [get this result], "variables": [ignored]}
    posts = {}
    for jsonData in jsonDataArray:
        jsonData = jsonData[10: -31]
        try:
            data = json.loads(jsonData)['data']['feedback']
            commentsInfo = data['display_comments']['edges']
            comments = []
            for comment in commentsInfo:
                comment = comment['node']
                if comment['body'] is not None:
                    comments.append({
                        'id': comment['id'],
                        'time': comment['created_time'],
#                         'author': comment['author'], # deep information of user
                        'message': comment['body']['text'],
                        'like': comment['feedback']['reactors']['count']
                    })
            actorId = data['owning_profile']['id']
            postId = data['share_fbid']
            posts['{}_{}'.format(actorId, postId)] = {
                'like': data['reaction_count']['count'],
                'share': data['share_count']['count'],
                'comment': data['display_comments_count']['count'],
                'commentsHighlight': comments
            }
        except:
            continue
    return posts

In [12]:
def getPostsFromContent(content, postInfos):
    soup = BeautifulSoup(content, 'html.parser')
    divs = soup.find_all('div', {'class': '_5pcr userContentWrapper'})
    posts = []
    for div in divs:
        try:
            posts_id = div.find_all('div', {'class': '_5pcp _5lel _2jyu _232_'})
            posts_time = div.find_all('span', {'class': 'fsm fwn fcg'})
            posts_link = div.find_all('a', {'class': '_5pcq'})
            posts_text = div.find_all('div', {'class': '_5pbx userContent _3576'})
            postIds = [re.search(r'_[0-9]+;[0-9]+;', div['id']).group()[1:-1].replace(';', '_') for div in posts_id]
            postTimes = [int(span.abbr['data-utime']) for span in posts_time]
            postLinks = ['https://www.facebook.com' + a['href'] for a in posts_link]
            postMessages = []
            for div in posts_text:
                for span in div.find_all('span', {'class': 'text_exposed_hide'}):
                    span.decompose()
                for span in div.find_all('span', {'class': 'see_more_link_inner'}):
                    span.decompose()
                postMessages.append(div.get_text())
            posts.append({
                'id': postIds[0],
                'time': postTimes[0],
                'link': postLinks[0],
                'message': postMessages[0],
                'like': postInfos[postIds[0]]['like'],
                'share': postInfos[postIds[0]]['share'],
                'comment': postInfos[postIds[0]]['comment'],
                'commentsHighlight': postInfos[postIds[0]]['commentsHighlight']
            })
        except:
#             print('Exception')
            continue
    return posts

In [15]:
pageNameOrId = 'khaosod' #'MorningNewsTV3' #'thairath' #'khaosod' 
source_url = 'https://www.facebook.com/pg/%s/posts/' % pageNameOrId
content = html.unescape(requests.get(source_url).text)
postInfos = getPostInfosFromContent(content)
posts = getPostsFromContent(content, postInfos)
urlResult = re.search(r'\/pages_reaction_units.*?unit_count=8', content).group()
# print(len(posts), posts[-1])
urlResult

'/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRrgsPACbXYbpx0ro3z1eJEcyGyofQK7HXEuYAFUyT5g7-AttoDzKGwhXqs74L9NasSV7Tee4w9Vi9raJN0USLXpUmdo09XY3Nk9h7q0KCFGEMXWKr-jx32Gv6fJIyVekB%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8'

In [ ]:
import datetime 
import time
now = datetime.datetime.now()
firstofjuly = now.replace(month=7,day=1,hour=0, minute=0, second=0, microsecond=0)
print(firstofjuly)
firstofjuly=int(time.mktime(firstofjuly.timetuple()))
print(firstofjuly)

testdate = now.replace(month=9,day=17,hour=0, minute=0, second=0, microsecond=0)
print(int(time.mktime(testdate.timetuple())))

In [ ]:
# for i in range(5):
  while True:
    try:
        next_url = 'https://www.facebook.com%s&__a=1' % urlResult
        tmp_content = requests.get(next_url).text[9:]
        next_content = json.loads(tmp_content)
        content = html.unescape(next_content['domops'][0][3]['__html'])
        postInfos = getPostInfosFromContent(tmp_content)
        next_posts = getPostsFromContent(content, postInfos)
        filtered_post = [next_post for next_post in next_posts if next_post['time'] > firstofjuly]
        out_post = [next_post for next_post in next_posts if next_post['time'] <= firstofjuly]
        posts += filtered_post
        if len(filtered_post) < len(next_posts):
            print('End by date')
            break
        try:
            urlResult = re.search(r'\/pages_reaction_units.*?unit_count=8', content).group()
        except:
            print('end')
            break
    except:
        print(next_url)
        continue
print(len(posts), posts[-1])

In [ ]:
# for i in range(5):
while True:
    try:
        next_url = 'https://www.facebook.com%s&__a=1' % urlResult
        tmp_content = requests.get(next_url).text[9:]
        next_content = json.loads(tmp_content)
        content = html.unescape(next_content['domops'][0][3]['__html'])
        postInfos = getPostInfosFromContent(tmp_content)
        next_posts = getPostsFromContent(content, postInfos)
        posts += next_posts
        try:
            urlResult = re.search(r'\/pages_reaction_units.*?unit_count=8', content).group()
            print(urlResult)
        except:
            print('end')
            break
    except:
        print(next_url)
        continue
print(len(posts), posts[-1])

/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRwjQt8lPxNrlQgNdtk0GHOWNUe9T8Yv6ZpTYNBAHwHJdnAoViNhAZdkHsit6Tc5DMiGGkDfBw1o8LroUtm8ResfaHtW0cJmHEUAa_Nkvq1ZWlnUxEF-Ax-q2J6YPXLn5e%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRCQx_3q3WUpPHfq2I-lk2vzjYjZUkwpAiXK2c3sPH_fn17WEf6UiEKZhQlaWFOuSJ1T_Xk24Y7HYpoDwdBeAHSbwJ6LNCff9meznikz04gmNCJvsNbLdhOlQykCXCGSRf%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRZ5_hIEKtgeOk4rAq_Ng3xm9dh_rHezou6cD5edF8ZAUBPB1ICzyG2Z69_RxCAtAbdJrj8X1EV7RzaY9h1pY82RFG7szke4rz_MOjLmDPR-1w2pTSGujLiZHFX5LvFnDT%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_cou

/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRjN8ZUYhOsHrjkFlHNn77EtNFxPFQRVaFIzcyKxDP2v_sqvsYEEUgATGcyTd7ryAYq2ZAdYJ8lsD7lIWSsbw1XB4s-8f_EHzFdVDRxEulk9K7w-M78T0lM07BH9U5f_eR%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRyUY-wGEsiAHBWa17n4jzM4bPi_y8hdq3YzFXAbSkBlLX_0KKTwKcrO0oNn6tAubeDB3poXXjbCbbDM7TV2baZLvZv4z2Yi9mts6Qg_4T3qx1w9gHhKwZUQ-wRWc1VEdw%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRCpwK3r6Y6ugs5FAdwOctk7rnjLIStROuL1W38uCRHDbKG1-ynJus3uiWGS30v68IpDmPadBw4Cn0k1d_JThWwX9P85v23Ft709bjRJqxpHnvsM8uTKGPoVYbatP8SKW8%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_cou

/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHREjgvT_2xwFBCKSCtjZbDlRgCXzv6xM-ajsycusuZMfRk5KbtTSwZB744iJZ-jmzAqFBRnIBTIuxFELPVrDXlPlAn-YnH7Qw5OpFBIhbfFWBo6x1NwLE_lZUktrGZOZk7%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRZZIlK8uLxrQNdGb5UHviygv1uhiirW0uByp2deJuLVcY3AIy6belFrpCNi9o5DFaLe4Jwn8cpjOe3WNHd8N1ggNLVGPn1mXY399YZ3G8iH7FWYVs6yvFgPwHfS4GAr2Q%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHROhmELdmREKcjD2QbSU0vKUEfJBQO6oHNWJAgP2kheuunOehbN3OeDbYLBzAvZn5xPMZc_7P7Bs-suY68qFv_b3tO-flqPerH5BvLc1RzGWLHUbRu4Q4-HrZ-Qrp-BTD0%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_cou

/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRrqLMjtLO8zeci12fkYGBpJY-5MnRMA-A1hEoZfwvWUMe1pRQPQUAsixmoAL6iQw_2-sGlsyDno0Zz2hMycoiwCsLCmvZHW2zeMCqyrgNHkzkaPIPz6YQNH3wJA5zhVnR%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRQekthk4bWw9kNeFu7paoh7NFERXup_0B29fAKln5DrvkyGpCu5c6YKCoYuZRJifV-ZlfXMtTsZF_ClUovS5CAnMStRtYFq8WhV82SCJwitnkP4Kn0PLawY87XVGmrAe_%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRvKDQoGTGlU_WlxF7V3RP93akzrsCPOKa7UJRbwBc9s-b_nVy1hbRo3L18r5a04BxEokexENqdtmWHknwtjn977eF8KSqF9-oT1lGLlI5iSYn8bwf8MawAZcM8LdF7hB8%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_cou

https://www.facebook.com/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRSUROsQ5LzKW4TM67hQsmo0vKDGBaVNFIbINPG1KRQfT-F7xlA83lOlFty5Rap_ugB-FSZGqHf56LoUkYsC0J7oimnSZR1G-HXTc0Ai_L0oCrmywv34DEW6gqukZSFi4e%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8&__a=1
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRbY9XBPr9mJ5sy6SclYc6hsrOMkQWIu_yMATKws9oYPjajyi4op7Ftf_becJzRLT2_DbpbjCBdGPsnSXZTxkWL6o38LiR8Mlg9rm6wNJ-oaJL_UOWdP872Tn9iCNxVg4x%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRJmUMQXUWB6ibby2gGBEtIKA3skWntMeaWz_AT4KUacVZIHm6EqvMPmDztmq-BD60QyaxqVeCCOLvBJmpz_iUuMAvvB6-SYDQ38LDAL1ldMxaopyutD4FS8z9U6d1kpyh%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&su

In [ ]:
content

# TEST

In [19]:
def getPostInfosFromContent(content):
    jsonDataArray = re.findall(r'\{\"result\"\:\{\"data.*?,\"variables', content)
    #ex. {"result": [get this result], "variables": [ignored]}
    posts = {}
    for jsonData in jsonDataArray:
        jsonData = jsonData[10: -31]
        try:
            data = json.loads(jsonData)['data']['feedback']
            commentsInfo = data['display_comments']['edges']
            comments = []
            for comment in commentsInfo:
                comment = comment['node']
                if comment['body'] is not None:
                    comments.append({
                        'id': comment['id'],
                        'time': comment['created_time'],
#                         'author': comment['author'], # deep information of user
                        'message': comment['body']['text'],
                        'like': comment['feedback']['reactors']['count']
                    })
            actorId = data['owning_profile']['id']
            postId = data['share_fbid']
            posts['{}_{}'.format(actorId, postId)] = {
                'like': data['reaction_count']['count'],
                'share': data['share_count']['count'],
                'comment': data['display_comments_count']['count'],
                'commentsHighlight': comments
            }
        except:
            continue
    return posts

In [ ]:
pageNameOrId = 'khaosod' #'MorningNewsTV3' #'thairath' #'khaosod' 
source_url = 'https://www.facebook.com/pg/%s/posts/' % pageNameOrId
content = html.unescape(requests.get(source_url).text)
postInfos = getPostInfosFromContent(content)
posts = getPostsFromContent(content, postInfos)
urlResult = re.search(r'\/pages_reaction_units.*?unit_count=8', content).group()
# print(len(posts), posts[-1])
urlResult

In [ ]:
re.search(r'\/pages_reaction_units.*?unit_count=8', content).group()

In [74]:
print(len(postInfos))
for a in postInfos:
    print(len(a))
    print(a)

19
32
129558990394402_4321889897827936
32
129558990394402_4321888641161395
32
129558990394402_4321645574519035
32
129558990394402_4321879734495619
32
129558990394402_4321834127833513
32
129558990394402_4321815914502001
32
129558990394402_4321785324505060
32
129558990394402_4321679427848983
32
129558990394402_4321771181173141
32
129558990394402_4321857194497873
32
129558990394402_4321739194509673
32
129558990394402_4321800784503514
32
129558990394402_4321753367841589
32
129558990394402_4321763544507238
32
129558990394402_4321722614511331
32
129558990394402_4321833217833604
32
129558990394402_4321727831177476
32
129558990394402_4321665404517052
32
129558990394402_4321701264513466
